### Aleksey Sepulveda y Jose Eduardo Sandoval


# Exercise 04

# Fraud Detection

## Introduction

- Fraud Detection Dataset from Microsoft Azure: [data](http://gallery.cortanaintelligence.com/Experiment/8e9fe4e03b8b4c65b9ca947c72b8e463)

Fraud detection is one of the earliest industrial applications of data mining and machine learning. Fraud detection is typically handled as a binary classification problem, but the class population is unbalanced because instances of fraud are usually very rare compared to the overall volume of transactions. Moreover, when fraudulent transactions are discovered, the business typically takes measures to block the accounts from transacting to prevent further losses. 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [2]:
import zipfile
with zipfile.ZipFile('../datasets/fraud_detection.csv.zip', 'r') as z:
    f = z.open('15_fraud_detection.csv')
    data = pd.io.parsers.read_table(f, index_col=0, sep=',')

In [3]:
data.head()

,accountAge,digitalItemCount,sumPurchaseCount1Day,sumPurchaseAmount1Day,sumPurchaseAmount30Day,paymentBillingPostalCode - LogOddsForClass_0,accountPostalCode - LogOddsForClass_0,paymentBillingState - LogOddsForClass_0,accountState - LogOddsForClass_0,paymentInstrumentAgeInAccount,ipState - LogOddsForClass_0,transactionAmount,transactionAmountUSD,ipPostalCode - LogOddsForClass_0,localHour - LogOddsForClass_0,Label
0,2000,0,0,0.00,720.25,5.064533,0.421214,1.312186,0.566395,3279.574306,1.218157,599.00,626.164650,1.259543,4.745402,0
1,62,1,1,1185.44,2530.37,0.538996,0.481838,4.401370,4.500157,61.970139,4.035601,1185.44,1185.440000,3.981118,4.921349,0
2,2000,0,0,0.00,0.00,5.064533,5.096396,3.056357,3.155226,0.000000,3.314186,32.09,32.090000,5.008490,4.742303,0
3,1,1,0,0.00,0.00,5.064533,5.096396,3.331154,3.331239,0.000000,3.529398,133.28,132.729554,1.324925,4.745402,0
4,1,1,0,0.00,132.73,5.412885,0.342945,5.563677,4.086965,0.001389,3.529398,543.66,543.660000,2.693451,4.876771,0


In [4]:
data.shape, data.Label.sum(), data.Label.mean()

((138721, 16), 797, 0.0057453449730033666)

In [5]:
X = data.drop(['Label'], axis=1)
y = data['Label']

# Exercice 04.1

Estimate a Logistic Regression

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test =train_test_split(X,y,random_state=2)

In [9]:
# logistic regression model 
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
Mod1=logmodel.fit(x_train, y_train)
Mod1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
#Predictions
y_pred=Mod1.predict(x_test)

In [11]:
##Metrics F1

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('Accuracy',accuracy_score(y_test, y_pred))
print('Accuracy',precision_score(y_test, y_pred))
print('Recall',recall_score(y_test, y_pred))
print('F1 score', f1_score(y_test, y_pred))

Accuracy 0.9938294743519507
Accuracy 0.0
Recall 0.0
F1 score 0.0


# Exercice 04.2

Under-sample the negative class using random-under-sampling

Which is parameter for target_percentage did you choose?
How the results change?

**Only apply under-sampling to the training set, evaluate using the whole test set**

In [12]:
def UnderSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_0_new =  n_samples_1 / target_percentage - n_samples_1
    n_samples_0_new_per = n_samples_0_new / n_samples_0

    filter_ = y == 0

    np.random.seed(seed)
    rand_1 = np.random.binomial(n=1, p=n_samples_0_new_per, size=n_samples)
    
    filter_ = filter_ & rand_1
    filter_ = filter_ | (y == 1)
    filter_ = filter_.astype(bool)
    
    return X[filter_], y[filter_]

In [13]:
Fraude = (y_train == 1).sum()
No_Fraude = (y_train == 0).sum()
float(Fraude)/(float(Fraude)+float(No_Fraude))

0.005622837370242215

In [14]:
#Balanceo propuesto de clase Fraude 25%

Xb, yb = UnderSampling(x_train, y_train, target_percentage= 0.25)

Fraude = (yb == 1).sum()
No_Fraude = (yb == 0).sum()
float(Fraude)/(float(Fraude)+float(No_Fraude))

0.2496798975672215

In [15]:
#Modelo Logit Undersampling

logmodel2 = LogisticRegression()
Mod2=logmodel2.fit(Xb, yb)
Mod2

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
# Predictions

y_pred2=Mod2.predict(x_test)

In [17]:
#Metrics F1 y demás

print('Accuracy',accuracy_score(y_test, y_pred2))
print('precision',precision_score(y_test, y_pred2))
print('recall',recall_score(y_test, y_pred2))
print('f1 score', f1_score(y_test, y_pred2))

Accuracy 0.9689743663677518
precision 0.038461538461538464
recall 0.16981132075471697
f1 score 0.0627177700348432


# Exercice 04.3

Now using random-over-sampling

In [19]:
import random
def OverSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_1_new =  -target_percentage * n_samples_0 / (target_percentage- 1)

    np.random.seed(seed)
    filter_ = np.random.choice(X[y == 1].shape[0], int(n_samples_1_new))
    # filter_ is within the positives, change to be of all
    filter_ = np.nonzero(y == 1)[0][filter_]
    
    filter_ = np.concatenate((filter_, np.nonzero(y == 0)[0]), axis=0)
    
    return X.iloc[filter_], y.iloc[filter_]

In [20]:
#Fraude 25%
X_b2, y_b2 = OverSampling(x_train, y_train, target_percentage = 0.25)

Fraude = (y_b2 == 1).sum()
No_Fraude = (y_b2 == 0).sum()
float(Fraude)/(float(Fraude)+float(No_Fraude))

0.25

In [21]:
#Logit Model OVersampling

logmodel3 = LogisticRegression()
Mod3=logmodel3.fit(X_b2, y_b2)
Mod3

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [22]:
# Predictions

y_pred3=Mod3.predict(x_test)

In [23]:
#Metrics F1 y demás

print('Accuracy',accuracy_score(y_test, y_pred3))
print('precision',precision_score(y_test, y_pred3))
print('recall',recall_score(y_test, y_pred3))
print('f1 score', f1_score(y_test, y_pred3))

Accuracy 0.9675326547677403
precision 0.04208416833667335
recall 0.19811320754716982
f1 score 0.0694214876033058


# Exercice 04.4*
Evaluate the results using SMOTE

Which parameters did you choose?

In [24]:
def SMOTE_MANUAL(X, y, target_percentage=0.5, k=5, seed=None):
    
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()
    
    #New samples
    n_samples_1_new =  int(-target_percentage * n_samples_0 / (target_percentage- 1) - n_samples_1)
    
    # A matrix to store the synthetic samples
    new = np.zeros((n_samples_1_new, X.shape[1]))
    
    # Create seeds
    np.random.seed(seed)
    seeds = np.random.randint(1, 1000000, 3)
    
    # Select examples to use as base
    np.random.seed(seeds[0])
    sel_ = np.random.choice(y[y==1].shape[0], n_samples_1_new)
    
    # Define random seeds (2 per example)
    np.random.seed(seeds[1])
    nn__ = np.random.choice(k, n_samples_1_new)
    np.random.seed(seeds[2])
    steps = np.random.uniform(size=n_samples_1_new)  

    # For each selected examples create one synthetic case
    for i, sel in enumerate(sel_):
        # Select neighbor
        nn_ = nn__[i]
        step = steps[i]
        # Create new sample
        new[i, :] = X[y==1].iloc[sel] - step * (X[y==1].iloc[sel] - X[y==1].iloc[nn_])
    
    X = np.vstack((X, new))
    y = np.append(y, np.ones(n_samples_1_new))
    
    return X, y

In [25]:
#Fraude del 25% en rebalanceo

X_b3, y_b3 = SMOTE_MANUAL(x_train, y_train, target_percentage=0.25, k=5)
Fraude = (y_b3 == 1).sum()
No_Fraude = (y_b3 == 0).sum()
float(Fraude)/(float(Fraude)+float(No_Fraude))

0.25

In [26]:
#Logit Model OVersampling

logmodel4 = LogisticRegression()
Mod4=logmodel4.fit(X_b3, y_b3)
Mod4

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
# Predictions

y_pred4=Mod4.predict(x_test)

In [28]:
#Metrics F1 y demás

print('Accuracy',accuracy_score(y_test, y_pred4))
print('precision',precision_score(y_test, y_pred4))
print('recall',recall_score(y_test, y_pred4))
print('f1 score', f1_score(y_test, y_pred4))

Accuracy 0.9474640292955797
precision 0.030338389731621937
recall 0.24528301886792453
f1 score 0.05399792315680166


# Exercice 04.5

Estimate a Logistic Regression, GaussianNB, K-nearest neighbors and a Decision Tree **Classifiers**

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

Combine the classifiers and comment

In [31]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

models = {'LOG': LogisticRegression(),
          'DTC': DecisionTreeClassifier(),
          'GNB': GaussianNB(),
          'KNN': KNeighborsClassifier()}

In [32]:
for model in models.keys():
    models[model].fit(x_train, y_train)

In [33]:
# predict test for each model
y_pred5 = pd.DataFrame(index=x_test.index, columns=models.keys())
for model in models.keys():
    y_pred5[model] = models[model].predict(x_test)

In [41]:
#Metricas

for model in models.keys():
    print(model,'accuracy',accuracy_score(y_test,y_pred5[model]))
    print(model,'precision',precision_score(y_test,y_pred5[model]))
    print(model,'recall',recall_score(y_test,y_pred5[model]))
    print(model,'f1 score', f1_score(y_test,y_pred5[model]))
    print(model,'BetaScore', metrics.fbeta_score(y_test,y_pred5[model], beta=10))

    

LOG accuracy 0.9938294743519507
LOG precision 0.0
LOG recall 0.0
LOG f1 score 0.0
LOG BetaScore 0.0
DTC accuracy 0.9882933018079063
DTC precision 0.12403100775193798
DTC recall 0.1509433962264151
DTC f1 score 0.13617021276595745
DTC BetaScore 0.1506198154534439
GNB accuracy 0.9236469536633892
GNB precision 0.01704996034892942
GNB recall 0.2028301886792453
GNB f1 score 0.031455742501828816
GNB BetaScore 0.18307899839811143
KNN accuracy 0.9937141374239498
KNN precision 0.38461538461538464
KNN recall 0.04716981132075472
KNN f1 score 0.08403361344537816
KNN BetaScore 0.047583152737209075


In [37]:
LOG= LogisticRegression()
DTC= DecisionTreeClassifier()
GNB= GaussianNB()
KNN= KNeighborsClassifier()

In [43]:
# Combination models

eclf1 = VotingClassifier(estimators=[('Logistic',LOG), ('Tree',DTC), ('Gaussian',GNB),('Neighbors',KNN)], voting='hard')
eclf1 = eclf1.fit(x_train, y_train)
y_p_f =eclf1.predict(x_test)
auc =metrics.roc_auc_score(y_p_f,y_test)
print('El AUC es: ', auc)

El AUC es:  0.9345438093040694


C:\Users\Asus GL553V\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Exercice 04.6

Using the under-sampled dataset

Evaluate a RandomForestClassifier and compare the results

change n_estimators=100, what happened

In [49]:
from sklearn.ensemble import RandomForestClassifier

RFmodel = RandomForestClassifier()
Mod6=RFmodel.fit(Xb, yb)

In [50]:
#Predictions

y_pred6=Mod6.predict(x_test)

In [51]:
#Metricas desempeño

print('Accuracy',accuracy_score(y_test, y_pred6))
print('precision',precision_score(y_test, y_pred6))
print('recall',recall_score(y_test, y_pred6))
print('f1 score', f1_score(y_test, y_pred6))



Accuracy 0.930394163951
precision 0.04125
recall 0.466981132075
f1 score 0.0758039816233


In [52]:
#Cambiando a 100 n estimators

RFmodel100 = RandomForestClassifier(n_estimators=100)
Mod7=RFmodel100.fit(Xb, yb)

In [53]:
#Predictions

y_pred7=Mod7.predict(x_test)

In [54]:
#Metricas desempeño

print('Accuracy',accuracy_score(y_test, y_pred7))
print('precision',precision_score(y_test, y_pred7))
print('recall',recall_score(y_test, y_pred7))
print('f1 score', f1_score(y_test, y_pred7))

Accuracy 0.931691704391
precision 0.0439746300211
recall 0.490566037736
f1 score 0.0807140085371
